In [1]:
def simple_coroutine():
    print('-> coroutine starte')
    x = yield
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()

In [3]:
my_coro

<generator object simple_coroutine at 0x000001A74FBD7748>

In [4]:
next(my_coro)

-> coroutine starte


In [5]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [6]:
my_coro2 = simple_coroutine()

In [7]:
my_coro2.send(123)

TypeError: can't send non-None value to a just-started generator

In [8]:
my_coro2.send(None)

-> coroutine starte


In [9]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [10]:
my_cc = simple_coro2(14)

In [11]:
from inspect import getgeneratorstate

In [12]:
getgeneratorstate(my_cc)

'GEN_CREATED'

In [13]:
next(my_cc)

-> Started: a = 14


14

In [14]:
getgeneratorstate(my_cc)

'GEN_SUSPENDED'

In [15]:
my_cc.send(28)

-> Received: b = 28


42

In [16]:
my_cc.send(99)

-> Received: c = 99


StopIteration: 

In [17]:
getgeneratorstate(my_cc)

'GEN_CLOSED'

In [30]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [19]:
coro_avg = averager()

In [20]:
next(coro_avg)

In [22]:
coro_avg.send(10)

10.0

In [23]:
coro_avg.send(30)

20.0

In [24]:
coro_avg.send(5)

15.0

In [26]:
from functools import wraps

def coroutine(func):
    """装饰器：向前执行到第一个yield表达式，预激func"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [32]:
coro_avg = averager()

In [33]:
from inspect import getgeneratorstate

In [34]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [35]:
coro_avg.send(10)

10.0

In [36]:
coro_avg.send(30)

20.0

In [37]:
coro_avg.send(5)

15.0

In [38]:
coro_avg = averager()

In [39]:
coro_avg.send(40)

40.0

In [40]:
coro_avg.send(50)

45.0

In [41]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [44]:
class DemoException(Exception):
    """为这次演示定义的异常类型"""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))

    raise RuntimeError('This line should never run.')

In [45]:
exc_coro = demo_exc_handling()

In [46]:
next(exc_coro)

-> coroutine started


In [47]:
exc_coro.send(11)

-> coroutine received: 11


In [48]:
exc_coro.send(22)

-> coroutine received: 22


In [49]:
exc_coro.close()

In [50]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [51]:
exc_coro = demo_exc_handling()

In [52]:
next(exc_coro)

-> coroutine started


In [53]:
exc_coro.send(11)

-> coroutine received: 11


In [54]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [55]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [56]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [ ]:
class DemoException(Exception):
    """为这次演示定义的异常类型"""
    
def demo_exc_handling():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending...')

    raise RuntimeError('This line should never run.')